In [23]:
import numpy as np
import json
import os

def generate_nodes(N, space_size=100, filename="nodes.json"):
    
    #Sinh N node cảm biến ngẫu nhiên trong không gian 3D và lưu vào folder input_data
    
    np.random.seed(0)  # để kết quả lặp lại
    node_positions = np.random.rand(N, 3) * space_size

    data = []
    for i in range(N):
        data.append({
            "id": i,
            "x": float(node_positions[i][0]),
            "y": float(node_positions[i][1]),
            "z": float(node_positions[i][2])
        })

    # Tạo thư mục lưu kết quả
    os.makedirs("input_data", exist_ok=True)
    filepath = f"input_data/{filename}"
    with open(filepath, "w") as f:
        json.dump(data, f, indent=4)

    print(f"Đã tạo file {filepath} chứa {N} node.")

# Sinh dữ liệu
for N in [100, 200, 500, 1000]:
    generate_nodes(N, space_size=100, filename=f"nodes_{N}.json")


Đã tạo file input_data/nodes_100.json chứa 100 node.
Đã tạo file input_data/nodes_200.json chứa 200 node.
Đã tạo file input_data/nodes_500.json chứa 500 node.
Đã tạo file input_data/nodes_1000.json chứa 1000 node.


In [28]:
import numpy as np
from sklearn.cluster import KMeans
import json

def cluster_split(nodes, node_ids, R=1, max_depth=10, depth=0):
    """
    Hàm phân cụm lặp theo Algorithm 1
    nodes: tọa độ (x, y, z) của các node
    node_ids: list ID tương ứng của các node
    R: ngưỡng tối tối đa số node trong 1 cụm
    max_depth: độ sâu đệ quy tối đa
    """
    if len(nodes) <= R or depth >= max_depth:
        center = np.mean(nodes, axis=0)
        return [{
            "node_ids": node_ids,
            "nodes": nodes,
            "center": center
        }]

    # Kmeans với k=2 để chia cụm
    kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
    labels = kmeans.fit_predict(nodes)

    clusters = []
    for i in range(2):
        sub_nodes = nodes[labels == i]
        sub_ids = [node_ids[j] for j in range(len(node_ids)) if labels[j] == i]
        clusters += cluster_split(sub_nodes, sub_ids, R, max_depth, depth + 1)

    return clusters


def choose_cluster_head(cluster):
    # Chọn cluster head là node gần tâm cụm nhất
    nodes = cluster["nodes"]
    center = cluster["center"]
    node_ids = cluster["node_ids"]

    dists = np.linalg.norm(nodes - center, axis=1)
    ch_index = np.argmin(dists)
    cluster_head = node_ids[ch_index]
    return cluster_head
 


In [34]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

input_folder = "input_data"
output_folder = "output_data"
os.makedirs(output_folder, exist_ok=True)
draw_folder = "draw_output"
os.makedirs(draw_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.startswith("nodes_") and filename.endswith(".json"):
        # Lấy số lượng node từ tên file
        number = filename.split("_")[1].split(".")[0]
        # Đọc dữ liệu từ file input
        with open(os.path.join(input_folder, filename), "r") as f:
            data = json.load(f)
        node_positions = np.array([[d["x"], d["y"], d["z"]] for d in data])
        node_ids = [d["id"] for d in data]
        # Phân cụm
        clusters_raw = cluster_split(node_positions, node_ids, R=5)
        # Tạo output
        clusters_output = {}
        for i, c in enumerate(clusters_raw):
            ch = choose_cluster_head(c)
            clusters_output[i] = {
                "nodes": c["node_ids"],
                "center": tuple(np.round(c["center"], 2)),
                "cluster_head": int(ch)
            }
        # Xuất ra file
        out_path = os.path.join(output_folder, f"nodes_{number}.json")
        with open(out_path, "w") as f:
            json.dump(clusters_output, f, indent=4)
        print(f"Đã xuất file {out_path}")
        # Vẽ và lưu hình
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')
        colors = plt.cm.get_cmap('tab10', len(clusters_output))
        for cid, info in clusters_output.items():
            nodes = np.array([node_positions[nid] for nid in info['nodes']])
            ax.scatter(nodes[:, 0], nodes[:, 1], nodes[:, 2],
                        label=f'Cụm {cid}',
                        color=colors(cid))
            ch_pos = node_positions[info['cluster_head']]
            ax.scatter(ch_pos[0], ch_pos[1], ch_pos[2],
                        color=colors(cid),
                        marker='*', s=20, edgecolor='k')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.legend()
        plt.title(f'Phân cụm các node cảm biến ({number} node)')
        ax.view_init(elev=30, azim=30)
        plt.tight_layout()
        draw_path = os.path.join(draw_folder, f"nodes_{number}.png")
        plt.savefig(draw_path)
        plt.close(fig)
        print(f"Đã lưu hình vẽ {draw_path}")

Đã xuất file output_data\nodes_100.json


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17944\1773477817.py:41: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab10', len(clusters_output))


Đã lưu hình vẽ draw_output\nodes_100.png
Đã xuất file output_data\nodes_1000.json


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17944\1773477817.py:41: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab10', len(clusters_output))
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17944\1773477817.py:57: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


Đã lưu hình vẽ draw_output\nodes_1000.png
Đã xuất file output_data\nodes_200.json


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17944\1773477817.py:41: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab10', len(clusters_output))


Đã lưu hình vẽ draw_output\nodes_200.png
Đã xuất file output_data\nodes_500.json


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17944\1773477817.py:41: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab10', len(clusters_output))
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17944\1773477817.py:57: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


Đã lưu hình vẽ draw_output\nodes_500.png
